In [ ]:
# Bibliotecas necessárias
!pip install -U sentence-transformers faiss-cpu pandas openai

In [ ]:
import pandas as pd

# Caminho JSON lOCAL
caminho_json = r'C:\Users\Documents\Conciliacao\Dados\dados_processos_anonymous_14052025.json'

# Carrega os dados
df = pd.read_json(caminho_json, lines=True)
df = df.dropna(subset=['inteiro_teor'])  # Remove registros sem texto
df.reset_index(drop=True, inplace=True)

print(f"{len(df)} documentos carregados.")
df[['numero_processo', 'classificacao']].head()

16583 documentos carregados.


,numero_processo,classificacao
0,5437274.43.2024.8.09.0012,frutifero
1,5437284.21.2024.8.09.0131,infrutifero
2,5438477.44.2024.8.09.0140,frutifero
3,5437321.31.2024.8.09.0169,infrutifero
4,5438510.45.2024.8.09.0007,infrutifero


In [ ]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import numpy as np

# Carrega o modelo
modelo_embedding = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Monta corpus com dados enriquecidos
corpus = []
for idx, row in df.iterrows():
    combinado = f"""
    [INTEIRO_TEOR]
    {row['inteiro_teor']}

    [Polo Passivo]: {row.get('polo_passivo', '')}
    [Tipo Parte Passiva]: {row.get('cpf_cnpj_polo_passivo', '')}
    [Assunto]: {row.get('assunto', '')}
    [Classe]: {row.get('classe', '')}
    """.strip()
    corpus.append(combinado)

# Salva esse texto no próprio DataFrame
df['texto_embedding'] = corpus

# Geração em batch
batch_size = 16
embeddings = []

for i in tqdm(range(0, len(corpus), batch_size), desc="Gerando embeddings"):
    batch = corpus[i:i + batch_size]
    emb = modelo_embedding.encode(batch, show_progress_bar=False)
    embeddings.extend(emb)

embeddings = np.array(embeddings)
print(f"Embeddings gerados: {embeddings.shape}")

In [ ]:
# Salvar os embeddings
np.save("embeddings_anonymous.npy", embeddings)

# Salvar os dados completos
df.to_csv("dados_processos_completo.csv", index=False, encoding='utf-8')
print("Dados e embeddings salvos.")

Dados e embeddings salvos.


In [9]:
import faiss

# Indexação com FAISS
embedding_dim = embeddings.shape[1]
embeddings_np = embeddings.astype('float32')

index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings_np)

print(f"Index FAISS criado com {index.ntotal} vetores.")

Index FAISS criado com 16583 vetores.


In [ ]:
import openai

# Cliente Maritaca
client = openai.OpenAI(
    api_key="COLOQUE_SUA_API_KEY_AQUI",
    base_url="https://chat.maritaca.ai/api"
)

def classificar_peticao(texto, k=10):
    # Gera embedding do novo texto
    emb_novo = modelo_embedding.encode([texto])[0].astype('float32')

    # Recupera documentos similares no índice FAISS
    _, indices = index.search(np.array([emb_novo]), k)
    docs_similares = df.iloc[indices[0]]['inteiro_teor'].tolist()
    contexto = "\n\n---\n\n".join(docs_similares)

    # Prompt completo da área de negócio
    prompt = f"""
Você é um assistente jurídico treinado para avaliar petições judiciais com base no seu conteúdo completo.
Sua tarefa é classificar o processo da seguinte forma:
    • Classificação: frutífero ou infrutífero
    • Justificativa: Explique de forma direta e fundamentada por que a ação tem ou não chance de realização de acordo (conciliação), com base nos critérios abaixo.

Avalie com base nos seguintes critérios (de forma técnica, objetiva e crítica):
    1. Parte ré (polo passivo):
        ◦ Se for uma empresa privada (CNPJ) e não houver provas documentais relevantes (como nota fiscal, contrato, prints, laudos, e-mails ou mensagens), classifique como infrutífero.
        ◦ Se for órgão público e o pedido envolver direitos sociais essenciais (como benefícios, saúde ou sustento), e houver documentos mínimos, pode ser frutífero. 
    2. Parte autora (polo ativo):
        ◦ Pessoa física em condição de hipossuficiência evidente (como trabalhador rural, doente, idoso, pessoa de baixa renda) aumenta a chance de êxito — classifique como frutífero se o pedido for coerente e verossímil. 
    3. Provas juntadas ou mencionadas:
        ◦ Ausência total de provas → infrutífero.
        ◦ Prints de conversa, contratos, comprovantes de pagamento, relatórios, prints de sistemas, laudos técnicos ou e-mails → aumentam a viabilidade do acordo.
    4. Urgência ou perigo da demora:
        ◦ Risco imediato à subsistência, moradia, renda, tratamento médico ou integridade física justifica uma avaliação positiva, mesmo com documentação limitada → frutífero.
    5. Valor da causa:
        ◦ Valor baixo com documentação consistente = frutífero.
        ◦ Valor baixo com pedido genérico ou sem provas = infrutífero.
        ◦ Valor alto, sem elementos probatórios suficientes, não altera a classificação → continua infrutífero.
    6. Clareza e estrutura do pedido:
        ◦ Petições com narrativa confusa, sem delimitação de pedidos ou argumentos dispersos → infrutífero.
        ◦ Petições com exposição clara dos fatos, pedidos objetivos e fundamentação direta → frutífero.
    7. Fundamentação jurídica e jurisprudência:
        ◦ Referências a jurisprudência pertinente, súmulas ou precedentes vinculantes reforçam a credibilidade do pedido → ponto positivo.
    8. Potencial de improcedência:
        ◦ Se a procedência depende de prova não anexada, inexistente ou de produção improvável, classifique como infrutífero.

Formato da resposta (obrigatório):
Classificação: frutífero ou infrutífero
Justificativa: [máximo 3 frases, linguagem direta e técnica]

Texto da nova petição:
{texto}

Outros documentos similares:
{contexto}

Resposta:
"""

    # Chamada à API Maritaca
    resposta = client.chat.completions.create(
        model="sabia-3.1",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1000
    )

    return resposta.choices[0].message.content.strip()

In [11]:
# Testar com um documento da própria base (ex: índice 125)
texto_teste = df.loc[21, 'inteiro_teor']
resposta = classificar_peticao(texto_teste)
print(resposta)

Classificação: frutífero

Justificativa: A parte autora é trabalhadora rural, reconhecidamente hipossuficiente e portadora de várias patologias graves que a incapacitam permanentemente para o trabalho, conforme atestado por profissionais médicos especialistas. A documentação médica apresentada e os laudos comprovam a incapacidade laborativa, justificando o pedido de restabelecimento de auxílio-doença rural e a concessão de aposentadoria por invalidez rural com adicional de 25%. Além disso, a urgência da medida é evidente devido ao caráter alimentar do benefício, essencial para a subsistência da autora e de sua família. A autora também demonstrou ter sido segurada especial, comprovando sua condição de trabalhadora rural sob regime de economia familiar, o que reforça a verossimilhança de suas alegações e justifica a concessão da tutela antecipada.


In [ ]:
import re
import unicodedata

def normalizar_texto(texto):
    texto = texto.lower().strip()
    return unicodedata.normalize('NFKD', texto).encode('ascii', errors='ignore').decode('utf-8')

def extrair_classificacao(resposta):
    resposta_norm = normalizar_texto(resposta)

    # Busca padrão "Classificação: frutifero" (com ou sem acento, com ou sem espaços)
    match = re.search(r'classificacao:\s*(frutifero|infrutifero)', resposta_norm)
    if match:
        return match.group(1)

    # Fallback por palavra solta
    if "infrutifero" in resposta_norm:
        return "infrutifero"
    elif "frutifero" in resposta_norm:
        return "frutifero"
    
    return "indefinido"

In [14]:
from tqdm import tqdm
import pandas as pd

def avaliar_llm_tempo_real(df, k=20, limite=100, seed=2024):
    resultados = []
    total = len(df) if limite is None else min(limite, len(df))
    acertos = 0

    # Amostragem aleatória com seed variável
    amostra_indices = df.sample(n=total, random_state=seed).index.tolist()

    for i, idx in enumerate(tqdm(amostra_indices, desc=f"Avaliando LLM (seed={seed})")):
        linha = df.loc[idx]
        texto = linha['texto_embedding']
        real = normalizar_texto(linha['classificacao'])

        try:
            resposta = classificar_peticao(texto, k=k)
            previsto = extrair_classificacao(resposta)
            match = previsto == real
            if match:
                acertos += 1

            resultados.append({
                "indice": idx,
                "real": real,
                "previsto": previsto,
                "match": match,
                "resposta_bruta": resposta
            })

            print(f"\n---\nResposta bruta da LLM:\n{resposta}\n")
            print(f"[{i+1}/{total}] Real: {real} | Previsto: {previsto} | Match: {match} | Acurácia parcial: {acertos / (i+1):.2%}")

        except Exception as e:
            resultados.append({
                "indice": idx,
                "real": real,
                "previsto": "erro",
                "match": False,
                "resposta_bruta": str(e)
            })
            print(f"[{i+1}/{total}] ERRO: {e} | Acurácia parcial: {acertos / (i+1):.2%}")

    df_resultado = pd.DataFrame(resultados)
    print(f"\nAcurácia final (seed {seed}): {df_resultado['match'].mean():.2%}")
    return df_resultado

In [ ]:
seeds = [42, 2023, 99, 7, 123]
resultados_acuracia = []

for seed in seeds:
    print(f"\n\nAvaliando com seed {seed}...\n")
    df_result = avaliar_llm_tempo_real(df, k=10, limite=100, seed=seed)
    
    acuracia = df_result['match'].mean()
    resultados_acuracia.append({
        "seed": seed,
        "acuracia": round(acuracia * 100, 2)
    })

In [17]:
df_avaliacoes = pd.DataFrame(resultados_acuracia)
print("\nAcurácia por seed:\n")
print(df_avaliacoes)


Acurácia por seed:

   seed  acuracia
0    42      52.0
1  2023      45.0
2    99      48.0
3     7      49.0
4   123      49.0
